# This file will preprocess the sentiment data (2020-2023) for BERT.

## Setting up the environment and getting all the data

In [ ]:
import pandas as pd
import numpy as np
import h5py
import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
DATA_NOT_SCRAPED = False #indicate whether we have to scrape data again. False since I've scraped it already, so we can read it directly.
PATH_sentiment = '/content/gdrive/MyDrive/lazard/consumer_comp_news_sentiment.csv'
PATH_comp_factors = '/content/gdrive/MyDrive/lazard/x_df_comp_factors.h5'
PATH_mapping = '/content/gdrive/MyDrive/lazard/rp_full_ticker_mapping.xlsx'
PATH_y = '/content/gdrive/MyDrive/lazard/y_series_return.h5'
PATH_price = '/content/gdrive/MyDrive/lazard/price_data.csv' # self-scraped data
LOOKBACK = 60
THRESHOLD = 2

In [ ]:
senti = pd.read_csv(PATH_sentiment) # 2020-2023, daily
senti.sort_values(by = ['RP_ENTITY_ID','TIMESTAMP_UTC'], inplace = True)

y_df = pd.read_hdf(PATH_y) # 2000-2023, monthly
y_df.sort_values(by = ['security','date'], inplace = True)
y_df = y_df.dropna(subset = ['MONTHLY_RETURN', "MONTHLY_RETURN_F1"])

cf = pd.read_hdf(PATH_comp_factors) # 2000-2023, monthly, optional
cf.sort_values(by = ['security','date'], inplace = True)


# only 2020-2023



In [ ]:
mapping_data= pd.read_excel(PATH_mapping)
ID2security = {}
security2ticker = {}
ID2ticker = {}

for i, row in mapping_data.iterrows():
  ID2security[row['RP_ENTITY_ID']] = row['security']
  security2ticker[row['security']] = row['ticker']
  ID2ticker[row['RP_ENTITY_ID']] = row['ticker']


picture，technical indicators, data put into different dfs.

In [ ]:
y_df.columns

Index(['security', 'date', 'MONTHLY_RETURN', 'MONTHLY_RETURN_F1'], dtype='object')

# Preprocess Data

## preprocess labels: remove missing labels

In [ ]:
print(y_df.shape)
y_df.dropna(subset=['MONTHLY_RETURN', 'MONTHLY_RETURN_F1'], inplace=True)
print(y_df.shape)


(6388, 4)
(6388, 4)


## Preprocess sentiment

In [ ]:
senti.columns


Index(['TIMESTAMP_UTC', 'RP_ENTITY_ID', 'ENTITY_NAME', 'RELEVANCE', 'TOPIC',
       'GROUP', 'TYPE', 'SUB_TYPE', 'EVENT_RELEVANCE', 'EVENT_SENTIMENT_SCORE',
       'EVENT_TEXT', 'HEADLINE'],
      dtype='object')

In [ ]:
# get datewise sentiment scores from the senti dataset
senti['security'] = [ID2security[ID] for ID in senti['RP_ENTITY_ID']]
senti.rename({'TIMESTAMP_UTC':'date'}, axis = 1, inplace = True)
senti['date'] = pd.to_datetime(senti['date']).dt.date
senti['month'] = pd.to_datetime(senti['date']).dt.strftime('%Y-%m')
senti['RELEVANT_SENTIMENT_SCORE'] = senti['EVENT_RELEVANCE'] * senti['EVENT_SENTIMENT_SCORE']
senti_datewise = senti.groupby(['security', 'date']).agg({'RELEVANT_SENTIMENT_SCORE':['mean','min','max']}).reset_index()
senti_datewise.columns = ["_".join(row) for row in senti_datewise.columns.ravel()]
senti_datewise.rename({'security_':'security', 'date_':'date'},axis =1,  inplace = True)
senti_datewise

<ipython-input-161-afc3fd2163bc>:8: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  senti_datewise.columns = ["_".join(row) for row in senti_datewise.columns.ravel()]


,security,date,RELEVANT_SENTIMENT_SCORE_mean,RELEVANT_SENTIMENT_SCORE_min,RELEVANT_SENTIMENT_SCORE_max
0,AMZN US Equity,2020-01-02,71.000000,71.0,71.0
1,AMZN US Equity,2020-01-03,-26.125000,-55.0,0.0
2,AMZN US Equity,2020-01-04,37.000000,37.0,37.0
3,AMZN US Equity,2020-01-05,37.000000,37.0,37.0
4,AMZN US Equity,2020-01-06,46.551351,0.0,56.0
...,...,...,...,...,...
13481,YUM US Equity,2023-10-20,-30.400000,-58.0,56.0
13482,YUM US Equity,2023-10-23,28.000000,0.0,56.0
13483,YUM US Equity,2023-10-24,46.000000,40.0,49.0
13484,YUM US Equity,2023-10-25,37.000000,37.0,37.0


In [ ]:
senti.columns

Index(['date', 'RP_ENTITY_ID', 'ENTITY_NAME', 'RELEVANCE', 'TOPIC', 'GROUP',
       'TYPE', 'SUB_TYPE', 'EVENT_RELEVANCE', 'EVENT_SENTIMENT_SCORE',
       'EVENT_TEXT', 'HEADLINE', 'security', 'month',
       'RELEVANT_SENTIMENT_SCORE'],
      dtype='object')

In [ ]:
senti = senti[['security','month', 'date', 'RELEVANT_SENTIMENT_SCORE','EVENT_TEXT', 'HEADLINE']]
senti

,security,month,date,RELEVANT_SENTIMENT_SCORE,EVENT_TEXT,HEADLINE
24,AMZN US Equity,2020-01,2020-01-02,71.0,"Silverleafe Capital Partners, LLC Buys Amazon....","Silverleafe Capital Partners, LLC Buys Amazon...."
41,AMZN US Equity,2020-01,2020-01-03,-55.0,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...
42,AMZN US Equity,2020-01,2020-01-03,-49.5,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...
48,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...
49,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...
...,...,...,...,...,...,...
55203,TGT US Equity,2023-10,2023-10-24,47.0,Target Promotes General Counsel Matt Zabel,Target Promotes General Counsel Matt Zabel to ...
55210,TGT US Equity,2023-10,2023-10-25,-53.0,Shipt lays off employees,"Shipt lays off employees, closes some open pos..."
55270,TGT US Equity,2023-10,2023-10-25,-53.0,Shipt lays off employees,'Difficult decision': Shipt lays off employees...
55338,TGT US Equity,2023-10,2023-10-26,-38.0,Target's COO and Chief External Engagement Off...,Two Target executives to retire


In [ ]:
def merge_text(text_series):
    return ' '.join(text_series)


In [ ]:
grouped_senti = senti.groupby(['security', 'month'])


In [ ]:
result = grouped_senti.agg({'EVENT_TEXT':merge_text, 'HEADLINE': merge_text}).reset_index()

In [ ]:
senti['date'] = pd.to_datetime(senti['date'])

In [ ]:
senti['str_senti'] = [str(round(i)) for i in senti['RELEVANT_SENTIMENT_SCORE']]

In [ ]:
senti['str_senti']

24        71
41       -55
42       -50
48         0
49         0
        ... 
55203     47
55210    -53
55270    -53
55338    -38
55471    -53
Name: str_senti, Length: 55488, dtype: object

In [ ]:
senti['text'] = senti['security'] + ' ' + senti['date'].dt.strftime('%Y-%m-%d')	 + ' ' + senti['str_senti'] + ' ' + senti['EVENT_TEXT'] + '. ' + senti['HEADLINE']

In [ ]:
senti['text'][0]

'CCL US Equity 2020-01-02 49 Galveston, Carnival Cruise Lines sign contract. Port Galveston, Carnival Cruise Lines sign contract to bring new state-of-the-art terminal'

# Train-Test Split
Data: sentiment, price and its extended features, y's extended features

### Take out rows in y_df (label data) where we have no features.

In [ ]:
y_df['month'] = y_df['date'].dt.strftime('%Y-%m')

In [ ]:
senti

,security,month,date,RELEVANT_SENTIMENT_SCORE,EVENT_TEXT,HEADLINE,str_senti,text
24,AMZN US Equity,2020-01,2020-01-02,71.0,"Silverleafe Capital Partners, LLC Buys Amazon....","Silverleafe Capital Partners, LLC Buys Amazon....",71,AMZN US Equity 2020-01-02 71 Silverleafe Capit...
41,AMZN US Equity,2020-01,2020-01-03,-55.0,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...,-55,AMZN US Equity 2020-01-03 -55 Alexa and Google...
42,AMZN US Equity,2020-01,2020-01-03,-49.5,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...,-50,AMZN US Equity 2020-01-03 -50 Alexa and Google...
48,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...,0,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...
49,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...,0,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...
...,...,...,...,...,...,...,...,...
55203,TGT US Equity,2023-10,2023-10-24,47.0,Target Promotes General Counsel Matt Zabel,Target Promotes General Counsel Matt Zabel to ...,47,TGT US Equity 2023-10-24 47 Target Promotes Ge...
55210,TGT US Equity,2023-10,2023-10-25,-53.0,Shipt lays off employees,"Shipt lays off employees, closes some open pos...",-53,TGT US Equity 2023-10-25 -53 Shipt lays off em...
55270,TGT US Equity,2023-10,2023-10-25,-53.0,Shipt lays off employees,'Difficult decision': Shipt lays off employees...,-53,TGT US Equity 2023-10-25 -53 Shipt lays off em...
55338,TGT US Equity,2023-10,2023-10-26,-38.0,Target's COO and Chief External Engagement Off...,Two Target executives to retire,-38,TGT US Equity 2023-10-26 -38 Target's COO and ...


In [ ]:
y_df

,security,date,MONTHLY_RETURN,MONTHLY_RETURN_F1,month
1,AMZN US Equity,2000-02-29,0.066787,-0.027223,2000-02
2,AMZN US Equity,2000-03-31,-0.027223,-0.176299,2000-03
3,AMZN US Equity,2000-04-28,-0.176299,-0.124574,2000-04
4,AMZN US Equity,2000-05-31,-0.124574,-0.248380,2000-05
5,AMZN US Equity,2000-06-30,-0.248380,-0.170407,2000-06
...,...,...,...,...,...
6605,YUM US Equity,2023-05-31,-0.084578,0.076618,2023-05
6606,YUM US Equity,2023-06-30,0.076618,-0.006351,2023-06
6607,YUM US Equity,2023-07-31,-0.006351,-0.060216,2023-07
6608,YUM US Equity,2023-08-31,-0.060216,-0.034318,2023-08


In [ ]:
# inner join y and X data to eliminate the rows where there is no features
y_df['month'] = y_df['date'].dt.strftime('%Y-%m')
X_y = pd.merge(senti[['security',	'month']], y_df, on = ['security', 'month'])
X_y.sort_values(by = ['security', 'month','date'], inplace = True)
print(f'before dropping dup: size y = {len(X_y)}')
X_y.drop_duplicates(inplace = True)
X_y.reset_index(inplace = True)
y = X_y
print(f'after dropping dup: size y = {len(X_y)}')

before dropping dup: size y = 51969
after dropping dup: size y = 1079


#### decide if we want company data in the features:

In [ ]:
X = senti

In [ ]:
X.columns

Index(['security', 'month', 'date', 'RELEVANT_SENTIMENT_SCORE', 'EVENT_TEXT',
       'HEADLINE', 'str_senti', 'text'],
      dtype='object')

### get y_train and y_val

In [ ]:
# first, get the y_train and y_val, then later based on that I will get X_train and X_val
# Create empty lists to store data for each stock
y_train_list = []
y_val_list = []

# Group the data by 'security' column
grouped = y.groupby('security')

for security, data in grouped:
    # Sort the data by the time (date) column
    data = data.sort_values(by='date')

    # Calculate the index for the 70% point
    split_index = int(0.7 * len(data))

    # Split the data into training and validation sets
    y_train, y_val = data.iloc[:split_index], data.iloc[split_index:]

    # Append data for the current stock to the lists
    y_train_list.append(y_train)
    y_val_list.append(y_val)

# Concatenate the individual DataFrames for each stock
y_train = pd.concat(y_train_list)
y_val = pd.concat(y_val_list)

# Reset index for the final DataFrames
y_train.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)

# Print the shapes of the resulting DataFrames
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)


y_train shape: (742, 6)
y_val shape: (337, 6)


### get X_train, X_val
* by selecting rows whose date is ealier than the max date under the stock group in y_train and y_val

In [ ]:
# Then, split X into train and validation set.
y_train_range = y_train.groupby(['security'])['date'].max().reset_index()
map_y_train_range = {}
for i, row in y_train_range.iterrows():
  map_y_train_range[row['security']] = row['date']


In [ ]:
# X_train contains all the rows where the date is earlier than/equal to the max_date in y_train (stratified in terms of security)
X_train = X.groupby(['security']).apply(lambda x: x[x['date']<= map_y_train_range[x.name]]).reset_index(drop = True)
# X_validation contains all the rows where the date is later than the max_date in y_train (stratified in terms of security)
X_val =  X.groupby(['security']).apply(lambda x: x[x['date'] > map_y_train_range[x.name]]).reset_index(drop = True)

In [ ]:
X_train

,security,month,date,RELEVANT_SENTIMENT_SCORE,EVENT_TEXT,HEADLINE,str_senti,text
0,AMZN US Equity,2020-01,2020-01-02,71.0,"Silverleafe Capital Partners, LLC Buys Amazon....","Silverleafe Capital Partners, LLC Buys Amazon....",71,AMZN US Equity 2020-01-02 71 Silverleafe Capit...
1,AMZN US Equity,2020-01,2020-01-03,-55.0,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...,-55,AMZN US Equity 2020-01-03 -55 Alexa and Google...
2,AMZN US Equity,2020-01,2020-01-03,-49.5,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...,-50,AMZN US Equity 2020-01-03 -50 Alexa and Google...
3,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...,0,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...
4,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...,0,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...
...,...,...,...,...,...,...,...,...
33490,YUM US Equity,2022-07,2022-07-28,54.0,Waldron Private Wealth LLC increased its posit...,Waldron Private Wealth LLC Grows Holdings in Y...,54,YUM US Equity 2022-07-28 54 Waldron Private We...
33491,YUM US Equity,2022-07,2022-07-28,56.0,"Pizza Hut Inc.: The introduction of the new, e...",A PENNE FOR YOUR SAUCE: PIZZA HUT INTRODUCES N...,56,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...
33492,YUM US Equity,2022-07,2022-07-28,56.0,"Pizza Hut Inc.: The introduction of the new, e...",Press Release: A PENNE FOR YOUR SAUCE: PIZZA H...,56,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...
33493,YUM US Equity,2022-07,2022-07-28,56.0,"Pizza Hut Inc.: The introduction of the new, e...",A PENNE FOR YOUR SAUCE: PIZZA HUT INTRODUCES N...,56,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...


### check

In [ ]:
data_lengths = {
    'Dataset': ['X_train', 'X_val', 'y_train', 'y_val'],
    'Length': [len(X_train), len(X_val), len(y_train), len(y_val)]
}

length_df = pd.DataFrame(data_lengths)
length_df

,Dataset,Length
0,X_train,33495
1,X_val,21993
2,y_train,742
3,y_val,337


In [ ]:
def check_composition(X_train, X_val, y_train, y_val):
  datasets = {
      'X_train': X_train['security'].value_counts(),
      'X_val': X_val['security'].value_counts(),
      'y_train': y_train['security'].value_counts(),
      'y_val': y_val['security'].value_counts()
  }

  # Create a DataFrame from the datasets dictionary
  table = pd.DataFrame(datasets)
  return table

In [ ]:
table = check_composition(X_train, X_val, y_train, y_val)
table

,X_train,X_val,y_train,y_val
AMZN US Equity,8284,5315,30,14
APTV US Equity,575,304,30,13
AZO US Equity,466,300,30,13
BKNG US Equity,561,405,30,14
CCL US Equity,811,483,30,14
CMG US Equity,715,498,30,14
DG US Equity,638,550,29,13
EBAY US Equity,994,393,30,13
F US Equity,3899,2619,30,14
GM US Equity,4102,2629,30,14


In [ ]:
table.sum(axis = 0)

X_train    33495
X_val      21993
y_train      742
y_val        337
dtype: int64

In [ ]:
# safe copies
y_val_original = y_val
X_val_origianl = X_val

In [ ]:
y_val_original

,index,security,month,date,MONTHLY_RETURN,MONTHLY_RETURN_F1
0,8284,AMZN US Equity,2022-07,2022-07-29,0.270596,-0.060615
1,8799,AMZN US Equity,2022-08,2022-08-31,-0.060615,-0.108622
2,9252,AMZN US Equity,2022-09,2022-09-30,-0.108622,-0.093451
3,9584,AMZN US Equity,2022-10,2022-10-31,-0.093451,-0.057595
4,9874,AMZN US Equity,2022-11,2022-11-30,-0.057595,-0.129894
...,...,...,...,...,...,...
332,44897,YUM US Equity,2023-05,2023-05-31,-0.084578,0.076618
333,44938,YUM US Equity,2023-06,2023-06-30,0.076618,-0.006351
334,44989,YUM US Equity,2023-07,2023-07-31,-0.006351,-0.060216
335,45024,YUM US Equity,2023-08,2023-08-31,-0.060216,-0.034318


In [ ]:
X_val.columns

Index(['security', 'month', 'date', 'RELEVANT_SENTIMENT_SCORE', 'EVENT_TEXT',
       'HEADLINE', 'str_senti', 'text'],
      dtype='object')

## Feature-engineering

### Actually Compute the set of price features experimented earlier

In [ ]:
# save checkpoint before feature engineering
X_train_beforefe = X_train
X_val_beforefe = X_val
y_train_beforefe = y_train
y_val_beforefe = y_val

In [ ]:
table1 = check_composition(X_train_beforefe, X_val_beforefe, y_train_beforefe, y_val_beforefe)
table1

,X_train,X_val,y_train,y_val
AMZN US Equity,8284,5315,30,14
APTV US Equity,575,304,30,13
AZO US Equity,466,300,30,13
BKNG US Equity,561,405,30,14
CCL US Equity,811,483,30,14
CMG US Equity,715,498,30,14
DG US Equity,638,550,29,13
EBAY US Equity,994,393,30,13
F US Equity,3899,2619,30,14
GM US Equity,4102,2629,30,14


### Impute, Fill in Missing values (Feature Augmentation Done)

In [ ]:

def is_numeric(data,col):
    """
    input: df[column]
    output: boolean
    """
    if len(data[col].dropna()) == 0:
      print(f"all nan values in {col}")
    first_non_null_value = data[col].dropna().iloc[0]
    is_numeric = isinstance(first_non_null_value, (int, float))
    return is_numeric



def fill_missing_with_column_mean(df):
    # Iterate through each column in the DataFrame
    for col in df.columns:
        try:
          is_numeric(df,col)
        except:
          print('Error with, cannot test if it\'s numeric', col)
        if is_numeric(df,col):
            # Calculate the mean of the current column
            column_mean = df[col].mean()
            # Fill NaN values in the current column with the mean
            df[col].fillna(column_mean, inplace=True)
    return df

def view_stock_date(df):
  view = df.groupby('security').agg({'date':['min','max','count']})
  return view


def p(df):
  sns.set(style="whitegrid")
  plt.figure(figsize=(24, 6))

  # Extract year and month from the 'date' column and combine them into a new column 'Year-Month'
  df['Year-Month'] = df['date'].dt.strftime('%Y-%m')

  # Create a countplot to visualize the distribution of dates by "Year-Month"
  sns.countplot(x='Year-Month', data=df, palette="Blues")

  # Customize the plot
  plt.title("Date Distribution by Year-Month")
  plt.xlabel("Year-Month")
  plt.ylabel("Count")
  plt.xticks(rotation=45)  # Rotate x-axis labels for readability

  # Show the plot
  plt.show()


In [ ]:
# impute missing values: 0 is imputed for sentiment columns; the price on the previous day is imputed for price columns.
# Define the columns I want to fill with 0
zero_fill_cols = ['RELEVANT_SENTIMENT_SCORE_mean', 'RELEVANT_SENTIMENT_SCORE_min', 'RELEVANT_SENTIMENT_SCORE_max']
fill0 = {'RELEVANT_SENTIMENT_SCORE_mean': 0,
                        'RELEVANT_SENTIMENT_SCORE_min': 0,
                        'RELEVANT_SENTIMENT_SCORE_max': 0}
# on X_train
# Fill specified columns with 0 and others with the previous row's value
X_train = X_train.fillna(fill0)
# forwardfill: fill in the price on the previous day
X_train.fillna(method='ffill', inplace=True)

# on X_test
X_val = X_val.fillna(fill0)
# forwardfill: fill in the price on the previous day
X_val.fillna(method='ffill', inplace=True)



In [ ]:
# replace inf and -inf with np.nan
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)

In [ ]:
# fill the rest (features generated based on past y_values) with mean column value
X_train = fill_missing_with_column_mean(X_train) #here
X_val = fill_missing_with_column_mean(X_val)

In [ ]:
# Count the total number of null entries in X_train
total_null_entries_train = X_train.isna().sum().sum()

# Count the total number of null entries in X_test
total_null_entries_test = X_val.isna().sum().sum()

# Print the results
print("Total number of null entries in X_train:", total_null_entries_train)
print("Total number of null entries in X_test:", total_null_entries_test)


Total number of null entries in X_train: 0
Total number of null entries in X_test: 0


In [ ]:
null_counts = X_train.isna().sum()
columns_with_nulls = null_counts[null_counts > 0].index.tolist()
print(columns_with_nulls)

[]


In [ ]:
# checkpoint
X_train_afterfe = X_train
X_val_afterfe = X_val
y_train_afterfe = y_train
y_val_afterfe = y_val

In [ ]:
y_val_afterfe.isna().sum()

index                0
security             0
month                0
date                 0
MONTHLY_RETURN       0
MONTHLY_RETURN_F1    0
dtype: int64

### Outliers:

In [ ]:
def trend(df, df2, stock = 'AMZN US Equity', target = 'MONTHLY_RETURN'):
    """
    df: the y data
    df2: the price data or another y data
    """
    if df.equals(df2):
      comparison = 'y'
    else:
      comparison = 'price'

    # plot for each stock
    df = df.copy()
    df = df.sort_values(by=['security','date'])
    df = df[df['security'] == stock]

    df2 = df2.copy()
    df2 = df2.sort_values(by=['security','date'])
    df2 = df2[df2['security'] == stock]
    df2 = df2.replace([np.inf, -np.inf], np.nan)

    # use the same starting date
    df = df[df['date'] >= df2['date'].min()]
    df2 = df2[df2['date'] >= df['date'].min()]

    print(f'check size: df:{df.shape}; df2:{df2.shape}')

    # Normalize 'MONTHLY_RETURN' and use a separate plot for each column in df2
    ascaler = MinMaxScaler()
    scaled_target = ascaler.fit_transform(df[[target]].values)

    n_columns = len(df2.columns)
    colors = ['b', 'g', 'r', 'c', 'm', 'y', '0.5', '0.6', '0.7', '0.8', '0.9', '0.4',
              (0.5, 0.1, 0.8), (0.2, 0.6, 0.1), (0.8, 0.2, 0.6), (0.2, 0.6, 0.8),
              (0.7, 0.7, 0.2), (0.5, 0.2, 0.5), (0.2, 0.5, 0.5), (0.8, 0.8, 0.2),
              (0.5, 0.2, 0.8), (0.8, 0.2, 0.2), (0.2, 0.8, 0.2), '0.3', '0.4', '0.1', '0.2', '0.7']

    for col_index, col in enumerate(df2.columns):
        print(f'{col} is numeric? {is_numeric(df2, col)}')
        if is_numeric(df2,col) and col != 'date' and col!= 'month' and col != 'security' and col != 'index':
            # if df and df2 are the same, we are plotting y's features against y.
            # we have 2 labels : monthly average, and next monthly average
            # Do not use the features generated in the other label.
            if comparison == 'y':
                if (target == 'MONTHLY_RETURN' and 'F1' in col) or (target == 'MONTHLY_RETURN_F1' and 'F1' not in col):
                    continue
            bscaler = MinMaxScaler()
            scaled_feature = bscaler.fit_transform(df2[[col]].values)
            color = colors[col_index % len(colors) ] # Cyclically use colors


            plt.figure(figsize=(12, 6))
            print(f'lendf: {len(df)}; lendf2: {len(df2)}')
            plt.plot(df2['date'], scaled_feature, label=col, color=color)
            plt.plot(df['date'], scaled_target, label= target, color='blue')
            plt.title(f'{stock}: Feature {col} vs Target {target}')
            plt.xlabel('Date')
            plt.ylabel(f'Normalized Value of {col}')
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            plt.show()




def check_and_visualize_outliers(data, threshold=THRESHOLD, stock = 'AMZN US Equity'):
    """
    Check and visualize outliers for each column of the given DataFrame.

    Parameters:
        data (DataFrame): The input data.
        threshold (float): The threshold for identifying outliers (default is 2.0).

    Returns:
        None
    """
    outliers = []
    # get the data from a specific stock
    data = data[data['security'] == stock]

    for col in data.columns:
        d = data[[col]]
        if is_numeric(data,col) and col != 'date' and col!= 'month' and col != 'security' and col != 'index':
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR

            col_outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)][[col]]

            outliers.append(col_outliers)

            plt.figure(figsize=(8, 6))
            sns.boxplot(data=d, x=col)
            sns.stripplot(data=col_outliers, x=col, color='pink', size=3)

            plt.title(f'{stock}: Box Plot & Outliers for {col}')
            plt.show()


def is_numeric(data,col):
    """
    input: df[column]
    output: boolean
    """
    if len(data[col].dropna()) == 0:
      print(f"all nan values in {col}")
    first_non_null_value = data[col].dropna().iloc[0]
    is_numeric = isinstance(first_non_null_value, (int, float))
    return is_numeric

# replace outliers for one stock, assuming the data is filtered to be only for one stock
def replace_outliers_with_iqr(data, threshold=THRESHOLD):
  data = data.copy()
  for col_index, col in enumerate(data.columns):
    if is_numeric(data,col) and col != 'date' and col!= 'month' and col != 'security' and col != 'index':

      Q1 = data[col].quantile(0.25)
      Q3 = data[col].quantile(0.75)
      IQR = Q3 - Q1

      lower_bound = Q1 - threshold * IQR
      upper_bound = Q3 + threshold * IQR

      # Replace values below the lower bound with the lower bound
      data[col] = data[col].apply(lambda x: lower_bound if x < lower_bound else x)

      # Replace values above the upper bound with the upper bound
      data[col] = data[col].apply(lambda x: upper_bound if x > upper_bound else x)

  return data

# given a general df containing all securities, split into datasets for each stock and apply the function "f" to each of them
# return:
# new_df: data for all stocks
# sub_dfs: data for each stock
def iter_allstock(df, f):
  securities = list(df['security'].unique())
  sub_dfs = []
  each_stock = {}
  for security in securities:
    df_stock = df[df['security'] == security] # get stock data for each stock
    df_sub = f(df_stock)
    sub_dfs.append(df_sub)
    each_stock[security] = df_sub

  new_df = pd.concat(sub_dfs)
  return new_df, each_stock


In [ ]:
X_train, _ = iter_allstock(X_train, replace_outliers_with_iqr)
X_val, _ = iter_allstock(X_val, replace_outliers_with_iqr)

In [ ]:
X_train

,security,month,date,RELEVANT_SENTIMENT_SCORE,EVENT_TEXT,HEADLINE,str_senti,text
0,AMZN US Equity,2020-01,2020-01-02,71.0,"Silverleafe Capital Partners, LLC Buys Amazon....","Silverleafe Capital Partners, LLC Buys Amazon....",71,AMZN US Equity 2020-01-02 71 Silverleafe Capit...
1,AMZN US Equity,2020-01,2020-01-03,-55.0,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...,-55,AMZN US Equity 2020-01-03 -55 Alexa and Google...
2,AMZN US Equity,2020-01,2020-01-03,-49.5,Alexa and Google-enabled smart plugs are down,Alexa and Google-enabled smart plugs are down ...,-50,AMZN US Equity 2020-01-03 -50 Alexa and Google...
3,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...,0,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...
4,AMZN US Equity,2020-01,2020-01-03,0.0,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink XT2,Arlo Pro 3 vs. Nest Cam IQ Outdoor vs. Blink X...,0,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...
...,...,...,...,...,...,...,...,...
33490,YUM US Equity,2022-07,2022-07-28,54.0,Waldron Private Wealth LLC increased its posit...,Waldron Private Wealth LLC Grows Holdings in Y...,54,YUM US Equity 2022-07-28 54 Waldron Private We...
33491,YUM US Equity,2022-07,2022-07-28,56.0,"Pizza Hut Inc.: The introduction of the new, e...",A PENNE FOR YOUR SAUCE: PIZZA HUT INTRODUCES N...,56,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...
33492,YUM US Equity,2022-07,2022-07-28,56.0,"Pizza Hut Inc.: The introduction of the new, e...",Press Release: A PENNE FOR YOUR SAUCE: PIZZA H...,56,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...
33493,YUM US Equity,2022-07,2022-07-28,56.0,"Pizza Hut Inc.: The introduction of the new, e...",A PENNE FOR YOUR SAUCE: PIZZA HUT INTRODUCES N...,56,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...


In [ ]:
# now clean the columns
#X_train.drop(['date_y'], axis = 1, inplace = True) # drop the date from y engineered features (date of label)
#X_train.rename({'date_x':'date'}, axis = 1, inplace = True)

#X_val.drop(['date_y'], axis = 1, inplace = True) # drop the date from y engineered features (date of label)
#X_val.rename({'date_x':'date'}, axis = 1, inplace = True)

In [ ]:
#X_train['month'] = X_train['date'].dt.strftime('%Y-%m')
#X_val['month'] = X_val['date'].dt.strftime('%Y-%m')

In [ ]:

if 'index' in y_train.columns:
  y_train.drop(['index'],axis = 1, inplace = True)
if 'index' in y_val.columns:
  y_val.drop(['index'],axis = 1, inplace = True)

In [ ]:
y_df.isna().sum()

security             0
date                 0
MONTHLY_RETURN       0
MONTHLY_RETURN_F1    0
month                0
dtype: int64

In [ ]:
y_df[y_df['MONTHLY_RETURN'].isnull()]

,security,date,MONTHLY_RETURN,MONTHLY_RETURN_F1,month


In [ ]:
y_val

,security,month,date,MONTHLY_RETURN,MONTHLY_RETURN_F1
0,AMZN US Equity,2022-07,2022-07-29,0.270596,-0.060615
1,AMZN US Equity,2022-08,2022-08-31,-0.060615,-0.108622
2,AMZN US Equity,2022-09,2022-09-30,-0.108622,-0.093451
3,AMZN US Equity,2022-10,2022-10-31,-0.093451,-0.057595
4,AMZN US Equity,2022-11,2022-11-30,-0.057595,-0.129894
...,...,...,...,...,...
332,YUM US Equity,2023-05,2023-05-31,-0.084578,0.076618
333,YUM US Equity,2023-06,2023-06-30,0.076618,-0.006351
334,YUM US Equity,2023-07,2023-07-31,-0.006351,-0.060216
335,YUM US Equity,2023-08,2023-08-31,-0.060216,-0.034318


In [ ]:
y_train

,security,month,date,MONTHLY_RETURN,MONTHLY_RETURN_F1
0,AMZN US Equity,2020-01,2020-01-31,0.087064,-0.062214
1,AMZN US Equity,2020-02,2020-02-28,-0.062214,0.035021
2,AMZN US Equity,2020-03,2020-03-31,0.035021,0.268900
3,AMZN US Equity,2020-04,2020-04-30,0.268900,-0.012785
4,AMZN US Equity,2020-05,2020-05-29,-0.012785,0.129567
...,...,...,...,...,...
737,YUM US Equity,2022-03,2022-03-31,-0.033040,-0.012824
738,YUM US Equity,2022-04,2022-04-29,-0.012824,0.038116
739,YUM US Equity,2022-05,2022-05-31,0.038116,-0.065531
740,YUM US Equity,2022-06,2022-06-30,-0.065531,0.079552


In [ ]:
y_train.isna().sum()

security             0
month                0
date                 0
MONTHLY_RETURN       0
MONTHLY_RETURN_F1    0
dtype: int64

In [ ]:
y_val.isna().sum()

security             0
month                0
date                 0
MONTHLY_RETURN       0
MONTHLY_RETURN_F1    0
dtype: int64

### Generate X_values for each y label


#### join the y (monthly) and X data (daily) together.
#### since we have relatively small amount of data, I matched the stock return at the end of the month to be the label for very day in the month to augment the data size.  
#### because of this, the plot in later sections for the real-y will have monthly plateaued values (real y will appear to be more rigid than the predicted y.)

In [ ]:
Xy_train = pd.merge(X_train, y_train, on = ['security', 'month'], suffixes = ('_x', '_y'))
Xy_val = pd.merge(X_val, y_val, on = ['security', 'month'], suffixes = ('_x', '_y'))

In [ ]:
Xy_train.columns

Index(['security', 'month', 'date_x', 'RELEVANT_SENTIMENT_SCORE', 'EVENT_TEXT',
       'HEADLINE', 'str_senti', 'text', 'date_y', 'MONTHLY_RETURN',
       'MONTHLY_RETURN_F1'],
      dtype='object')

In [ ]:
print(f'check data:Xy_train shape: {Xy_train.shape}; Xy_val shape: {Xy_val.shape}')
Xy_train = Xy_train[Xy_train['date_x']<=Xy_train['date_y']]
Xy_val = Xy_val[Xy_val['date_x']<=Xy_val['date_y']]
print(f'check data:Xy_train shape: {Xy_train.shape}; Xy_val shape: {Xy_val.shape}')

check data:Xy_train shape: (33495, 11); Xy_val shape: (18447, 11)
check data:Xy_train shape: (33402, 11); Xy_val shape: (18383, 11)


In [ ]:
Xy_val.isna().sum()

security                    0
month                       0
date_x                      0
RELEVANT_SENTIMENT_SCORE    0
EVENT_TEXT                  0
HEADLINE                    0
str_senti                   0
text                        0
date_y                      0
MONTHLY_RETURN              0
MONTHLY_RETURN_F1           0
dtype: int64

In [ ]:
# now clean the columns
Xy_train.drop(['date_y'], axis = 1, inplace = True) # drop the date from y engineered features (date of label)
Xy_train.rename({'date_x':'date'}, axis = 1, inplace = True)

Xy_val.drop(['date_y'], axis = 1, inplace = True) # drop the date from y engineered features (date of label)
Xy_val.rename({'date_x':'date'}, axis = 1, inplace = True)

In [ ]:
Xy_val = Xy_val[['security',	'month',	'date', 'text','MONTHLY_RETURN','MONTHLY_RETURN_F1']]
Xy_train = Xy_train[['security',	'month',	'date', 'text','MONTHLY_RETURN','MONTHLY_RETURN_F1']]

In [ ]:
Xy_val.to_csv('/content/gdrive/MyDrive/lazard/preprocessed_data/Xy_valBERT.csv', index = False)
Xy_train.to_csv('/content/gdrive/MyDrive/lazard/preprocessed_data/Xy_trainBERT.csv', index = False)

In [ ]:
Xy_train

,security,month,date,text,MONTHLY_RETURN,MONTHLY_RETURN_F1
0,AMZN US Equity,2020-01,2020-01-02,AMZN US Equity 2020-01-02 71 Silverleafe Capit...,0.087064,-0.062214
1,AMZN US Equity,2020-01,2020-01-03,AMZN US Equity 2020-01-03 -55 Alexa and Google...,0.087064,-0.062214
2,AMZN US Equity,2020-01,2020-01-03,AMZN US Equity 2020-01-03 -50 Alexa and Google...,0.087064,-0.062214
3,AMZN US Equity,2020-01,2020-01-03,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...,0.087064,-0.062214
4,AMZN US Equity,2020-01,2020-01-03,AMZN US Equity 2020-01-03 0 Arlo Pro 3 vs. Nes...,0.087064,-0.062214
...,...,...,...,...,...,...
33490,YUM US Equity,2022-07,2022-07-28,YUM US Equity 2022-07-28 54 Waldron Private We...,0.079552,-0.092215
33491,YUM US Equity,2022-07,2022-07-28,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...,0.079552,-0.092215
33492,YUM US Equity,2022-07,2022-07-28,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...,0.079552,-0.092215
33493,YUM US Equity,2022-07,2022-07-28,YUM US Equity 2022-07-28 56 Pizza Hut Inc.: Th...,0.079552,-0.092215


In [ ]:
FINAL_FEATURES = []
for col in list(Xy_train.columns):
  if col != 'security' and col != 'date_x' and col !=  'month' and col != 'date_y':
    FINAL_FEATURES.append(col)